# Data preparation

In [32]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os
import sys

import tiktoken
import openai

from langdetect import detect, DetectorFactory

from tqdm.asyncio import tqdm
from tqdm.asyncio import tqdm as atqdm

import logging
import asyncio
import aiofiles
import time
import json
import re

In [33]:
load_dotenv()
client = openai.OpenAI()
client_async = openai.AsyncOpenAI()

csv = pd.read_csv('./jobs_data_clean.csv')

df = csv.copy()
df.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer
1,Austria,Europe,True,True,default,google.com,Senior Ios Developer,Pyramid Global Technologies,Austria,"Trabajo.org - Stellenangebote, Arbeit, StudySm...",A minimum of 6+ years of concurrent commercial...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTZW5pb3IgSW9zIERldmVsb3Blci...,2025-01-13 12:20:43 UTC,iOS developer
2,Austria,Europe,True,True,default,google.com,iOS Developer - Permanent remote,Bluestorm Recruitment by Dazzle,Austria,Jooble,iOS Developer\r\n\r\nOur client is a leading m...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIC0gUGVybW...,2025-01-13 12:20:43 UTC,iOS developer
3,Austria,Europe,True,True,default,google.com,Sr. Ios Developer,Bykon,Austria,"Trabajo.org - Stellenangebote, Arbeit",In ByKon we're looking for an exceptional Sr.\...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTci4gSW9zIERldmVsb3BlciIsIm...,2025-01-13 12:20:43 UTC,iOS developer
4,Austria,Europe,True,True,default,google.com,Software Engineer/ iOS,Bitpanda,Anywhere,GrabJobs,Who we are\r\n\r\nWe simplify wealth creation....,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBFbmdpbmVlci8gaU...,2025-01-13 12:20:43 UTC,iOS developer


## Determine the language of vacancies

In [51]:
def count_tokens(text, model="gpt-4"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

text = "en"
print("Number of tokens:", count_tokens(text, "gpt-4-turbo"))

Number of tokens: 1


In [9]:
df.loc[0, "Job Description"]

"We are a dynamic FinTech company headquartered in Australia, now expanding our footprint to Malaysia.\r\nSpecializing in innovative payment and remittance solutions.\r\n\r\nWhy Join Us?\r\n\r\nYoung & dynamic workplace & culture (with office recreational amenities provided) Progressive career prospects Well rewarding project contributions Responsibilities:\r\nDevelop, test, and deploy high quality mobile apps for Android/iOS with a focus on performance, scalability, and reliability.\r\nCollaborate with product managers, designers, and developers to deliver tailored FinTech/Payments/Remittance solutions.\r\nImplement responsive design, accessibility, and security best practices.\r\nStay updated on mobile development trends and integrate new tools to improve efficiency.\r\nParticipate in code reviews, debugging, and troubleshooting to maintain code quality.\r\nOptimize mobile development processes for productivity and workflow efficiency.\r\nDesign scalable mobile applications using nat

### `langdetect` library

In [34]:
DetectorFactory.seed = 0  # Фиксируем результат, чтобы не было случайных изменений

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

# Применяем к колонке Job Description
df["Language langdetect"] = df["Job Description"].apply(detect_language)

In [7]:
df.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language langdetect
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer,en
1,Austria,Europe,True,True,default,google.com,Senior Ios Developer,Pyramid Global Technologies,Austria,"Trabajo.org - Stellenangebote, Arbeit, StudySm...",A minimum of 6+ years of concurrent commercial...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTZW5pb3IgSW9zIERldmVsb3Blci...,2025-01-13 12:20:43 UTC,iOS developer,en
2,Austria,Europe,True,True,default,google.com,iOS Developer - Permanent remote,Bluestorm Recruitment by Dazzle,Austria,Jooble,iOS Developer\r\n\r\nOur client is a leading m...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIC0gUGVybW...,2025-01-13 12:20:43 UTC,iOS developer,en
3,Austria,Europe,True,True,default,google.com,Sr. Ios Developer,Bykon,Austria,"Trabajo.org - Stellenangebote, Arbeit",In ByKon we're looking for an exceptional Sr.\...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTci4gSW9zIERldmVsb3BlciIsIm...,2025-01-13 12:20:43 UTC,iOS developer,en
4,Austria,Europe,True,True,default,google.com,Software Engineer/ iOS,Bitpanda,Anywhere,GrabJobs,Who we are\r\n\r\nWe simplify wealth creation....,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBFbmdpbmVlci8gaU...,2025-01-13 12:20:43 UTC,iOS developer,en


In [35]:
df.columns

Index(['Location', 'Region', 'EU Member', 'Schengen Agreement',
       'Google Domain Type', 'Google Domain Used', 'Job Title', 'Company Name',
       'Job Location', 'Apply Options', 'Job Description', 'Work from home',
       'Salary', 'Schedule type', 'Qualifications', 'Job ID', 'Search Date',
       'Search Query', 'Language langdetect'],
      dtype='object')

In [36]:
df['Language langdetect'].value_counts()

Language langdetect
en       2445
de         93
it         79
es         68
nl         45
fr         36
cs         23
pt         21
pl         19
hu          8
sv          5
ru          4
ro          4
bg          2
fi          2
sk          2
no          1
uk          1
da          1
lv          1
zh-cn       1
hr          1
sl          1
Name: count, dtype: int64

In [38]:
test = df.head(400).copy()

In [45]:
# Configure logging
logging.basicConfig(level=logging.WARNING, format="%(asctime)s - %(levelname)s - %(message)s")

# Suppress OpenAI API HTTP logs
logging.getLogger("httpx").setLevel(logging.WARNING)



async def check_rate_limits(response_headers, rpm):
    """Checks OpenAI API rate limits and waits if needed, without processing reset time."""

    # Extract limits from headers
    limit_requests = int(response_headers.get("x-ratelimit-limit-requests", rpm))  # Default to given RPM if missing
    limit_tokens = int(response_headers.get("x-ratelimit-limit-tokens", 0))  # TPM is always provided

    remaining_requests = int(response_headers.get("x-ratelimit-remaining-requests", limit_requests))
    remaining_tokens = int(response_headers.get("x-ratelimit-remaining-tokens", limit_tokens))

    # If RPD or TPD is reached → Stop execution
    if remaining_requests == 0:
        print(f"🔴 Request limit (RPD) reached! Execution stopped.")
        return False  # Stop execution

    if remaining_tokens == 0:
        print(f"🔴 Token limit (TPD) reached! Execution stopped.")
        return False  # Stop execution

    # If RPM or TPM is reached → Wait 60 seconds and continue
    if remaining_requests < 1 or remaining_tokens < 1:
        print(f"🟡 API rate limit reached! Waiting 60 seconds...")
        await asyncio.sleep(60)

    return True  # Continue execution



async def chatgpt_async(
    input_column_name=None,
    output_column_name=None,
    input_text_length=None,
    output_text_length=5,
    num_rows=None,
    df=None,
    user_prompt=None,
    gpt_model=None,
    client=None,
    batch_size=None,
    cache_file=None,
    rpm=500,
    concurrency_limit=10,
    max_retries=3,
    retry_delay=0.121
):
    """Async function to process API requests in batches and cache responses."""

    # Input Validations
    if df is None:
        raise ValueError("df (dataframe) must be provided.")
    if input_column_name is None:
        raise ValueError("input_column_name must be provided.")
    if input_column_name not in df.columns:
        raise ValueError(f"{input_column_name} column does not exist in the dataframe.")
    if output_column_name is None:
        raise ValueError("output_column_name must be provided.")
    if gpt_model is None:
        raise ValueError("gpt_model must be provided.")
    if user_prompt is None:
        raise ValueError("user_prompt must be provided.")
    if client is None:
        raise ValueError("client must be provided.")

    # Prepare Data
    df = df.head(num_rows).copy() if num_rows else df.copy()

    if output_column_name not in df.columns:
        df[output_column_name] = ""

    # Load Cache if Enabled
    cache = {}
    if cache_file and os.path.exists(cache_file):
        async with aiofiles.open(cache_file, "r", encoding="utf-8") as f:
            try:
                cache = json.loads(await f.read())
            except json.JSONDecodeError:
                logging.warning("Cache file is corrupted or empty. Proceeding without cache.")

    semaphore = asyncio.Semaphore(concurrency_limit)  # Limit concurrent requests

    async def process_row(index, row):
        """Asynchronously process each row with caching and retry support."""
        async with semaphore:
            column_text = row[input_column_name]
            truncated_text = " ".join(column_text.split()[:input_text_length]) if input_text_length else column_text

            # Check if result is cached
            if cache_file and truncated_text in cache:
                df.at[index, output_column_name] = cache[truncated_text]
                return  # Skip API call

            prompt = f"{user_prompt} Text: {truncated_text}"

            for attempt in range(max_retries):
                try:
                    # Fetch response with raw headers
                    response = await client.chat.completions.with_raw_response.create(
                        model=gpt_model,
                        messages=[{"role": "user", "content": prompt}],
                        max_tokens=output_text_length,
                        temperature=0
                    )

                    # Check API limits using response headers
                    if not await check_rate_limits(response.headers, rpm):
                        return  # Stop processing if RPD/TPD is reached

                    response_text = response.parse().choices[0].message.content

                    if cache_file:
                        cache[truncated_text] = response_text
                    df.at[index, output_column_name] = response_text
                    return  

                except Exception as e:
                    if attempt < max_retries - 1:
                        #logging.warning(f"Error on row {index}, attempt {attempt+1}: {e}. Retrying in {retry_delay}s...")
                        await asyncio.sleep(retry_delay)
                    else:
                        df.at[index, output_column_name] = f"Error: {str(e)}"
                        logging.error(f"Final failure for row {index}: {e}")

    # Process in Batches
    total_rows = len(df)

    if batch_size and batch_size < total_rows:
        batch_indices = np.array_split(range(total_rows), len(df) // batch_size + 1)
        batches = [df.iloc[idx] for idx in batch_indices]
    else:
        batches = [df]

    with atqdm(total=len(batches), desc="Processing Batches", leave=True) as pbar:
        for batch in batches:
            tasks = [process_row(index, row) for index, row in batch.iterrows()]
            await asyncio.gather(*tasks)

            # Save cache after each batch
            if cache_file:
                async with aiofiles.open(cache_file, "w", encoding="utf-8") as f:
                    await f.write(json.dumps(cache, ensure_ascii=False, indent=4))

            pbar.update(1)  # Update progress bar

    return df

In [ ]:
df_result = await chatgpt_async(
    input_column_name="Job Description", 
    output_column_name="Language gpt-3.5-turbo-0125",
    input_text_length=50,
    output_text_length=1,
    num_rows=None,  
    df=df, 
    user_prompt="Detect the language of the text and return ONLY the ISO country code (e.g., en, fr, de, ect.).",
    gpt_model="gpt-3.5-turbo-0125",
    client=client_async,
    rpm=3500,
    batch_size=30,
    concurrency_limit=10,
    cache_file="./cache/language_cache.json"
)
df_result

Processing Batches: 100%|██████████| 96/96 [02:49<00:00,  1.76s/it] 


,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language langdetect,Language gpt-3.5-turbo-0125
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer,en,en
1,Austria,Europe,True,True,default,google.com,Senior Ios Developer,Pyramid Global Technologies,Austria,"Trabajo.org - Stellenangebote, Arbeit, StudySm...",A minimum of 6+ years of concurrent commercial...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTZW5pb3IgSW9zIERldmVsb3Blci...,2025-01-13 12:20:43 UTC,iOS developer,en,en
2,Austria,Europe,True,True,default,google.com,iOS Developer - Permanent remote,Bluestorm Recruitment by Dazzle,Austria,Jooble,iOS Developer\r\n\r\nOur client is a leading m...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIC0gUGVybW...,2025-01-13 12:20:43 UTC,iOS developer,en,en
3,Austria,Europe,True,True,default,google.com,Sr. Ios Developer,Bykon,Austria,"Trabajo.org - Stellenangebote, Arbeit",In ByKon we're looking for an exceptional Sr.\...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTci4gSW9zIERldmVsb3BlciIsIm...,2025-01-13 12:20:43 UTC,iOS developer,en,en
4,Austria,Europe,True,True,default,google.com,Software Engineer/ iOS,Bitpanda,Anywhere,GrabJobs,Who we are\r\n\r\nWe simplify wealth creation....,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBFbmdpbmVlci8gaU...,2025-01-13 12:20:43 UTC,iOS developer,en,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2858,United States,Northern America,False,False,default,google.com,Android Developer Direct Client -A,Whiztek Corp,"Houston, TX","Dice, Teal",Job Title: Android Developer\r\n\r\nLocation: ...,NaN,NaN,Contractor,NaN,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciBEaX...,2025-01-13 12:12:30 UTC,Android developer,en,en
2859,United States,Northern America,False,False,default,google.com,Android Lead Developer,Capgemini,"Woodland, CA","LinkedIn, SitePoint, WKRN Jobs, SaluteMyJob, B...",Job Title: Senior Android Lead Developer\r\n\r...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIExlYWQgRGV2ZWxvcG...,2025-01-13 12:12:30 UTC,Android developer,en,en
2860,United States,Northern America,False,False,default,google.com,IOS AND Android Developer @ TX/NJ/DE/NYC ( Ons...,Kaizen Technologies,"Monroe Township, NJ",Teal,About the position\r\n\r\nWe are seeking exper...,NaN,NaN,Full-time and Contractor,No degree mentioned,eyJqb2JfdGl0bGUiOiJJT1MgQU5EIEFuZHJvaWQgRGV2ZW...,2025-01-13 12:12:30 UTC,Android developer,en,en
2861,United States,Northern America,False,False,default,google.com,Sr. Android Developer,Globant,"San Mateo, CA","Adzuna, WKRN Jobs, Aspire Media Group Jobs, WA...",Mobile Android Developer\r\n\r\nJob Summary:\r...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTci4gQW5kcm9pZCBEZXZlbG9wZX...,2025-01-13 12:12:30 UTC,Android developer,en,en


In [49]:
df_result['Language gpt-3.5-turbo-0125'].value_counts()

Language gpt-3.5-turbo-0125
en                         2421
de                           96
it                           72
es                           61
nl                           48
fr                           34
cs                           29
pt                           25
pl                           19
hu                            8
ru                            7
ro                            6
sv                            5
es_ES                         4
hr                            4
it_IT                         3
es) con posibilidad           2
sk                            2
ar                            2
This text is in English       2
español                       2
fi                            2
zh                            1
lv                            1
it, it is Italian             1
it-IT                         1
sl                            1
es.                           1
da                            1
uk                            1
it (Italian)

### gpt-3.5-turbo-0125